In [1]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pandas as pd
from sklearn import preprocessing

from xgboost import plot_importance
import numpy as np
from sklearn.metrics import confusion_matrix


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [15]:
df = pd.read_csv("D:\\analyticsvidhyahackathon\\train_LZdllcl.csv")
df.set_index('employee_id',inplace=True)


In [16]:
dfvalidation = pd.read_csv("D:\\analyticsvidhyahackathon\\test_2umaH9m.csv")
dfvalidation.set_index('employee_id',inplace=True)
dfvalidation['is_promoted'] = 5
#dfvalidation.head()


In [17]:
##Merging the train and test dataset
dfmerged = pd.concat([df,dfvalidation])
dfmerged.shape

(78298, 13)

In [18]:
df['previous_year_rating'].describe()

count    50684.000000
mean         3.329256
std          1.259993
min          1.000000
25%          3.000000
50%          3.000000
75%          4.000000
max          5.000000
Name: previous_year_rating, dtype: float64

In [19]:
df['education'].describe()

count          52399
unique             3
top       Bachelor's
freq           36669
Name: education, dtype: object

In [8]:
## Filling missing enteris with maximum occuring event
dfmerged['previous_year_rating'].fillna(3.0, inplace=True)
dfmerged['education'].fillna('Bachelor\'s', inplace=True)


In [9]:
dfmerged['age'].describe()

count    78298.000000
mean        34.797619
std          7.665928
min         20.000000
25%         29.000000
50%         33.000000
75%         39.000000
max         60.000000
Name: age, dtype: float64

In [ ]:
### Dividing age into different bands as this company has 9 verticals and they are finding promotion for manager and below.So didvided
###age into 4 different bands
'''
def findBand(age):
    if age <=29:
        return 1
    elif age>29 and age<=33:
        return 2
    elif age>33 and age<=39:
        return 3
    else:
        return 4

dfmerged['band'] = dfmerged['age'].apply(findBand)
'''

In [6]:
##One hot encoding
dfmerged = pd.concat([dfmerged[['no_of_trainings','age','previous_year_rating','length_of_service','KPIs_met >80%','awards_won?','avg_training_score','is_promoted']],
               pd.get_dummies(dfmerged['gender'],drop_first = True),pd.get_dummies(dfmerged['education'],drop_first = True),pd.get_dummies(dfmerged['recruitment_channel'],drop_first = True),pd.get_dummies(dfmerged['department'],drop_first = True),pd.get_dummies(dfmerged['region'],drop_first = True)],axis=1)


In [ ]:
###Generating all possible pair of interactions between 2 pair of columns.
##Then removing those columns 

from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

Y = dfmerged.is_promoted
X = dfmerged.drop(['is_promoted'],1)
 


def add_interactions(df):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns)+['_'.join(x) for x in combos]
    
    #scaler = MinMaxScaler()
    #scaler.fit(df)
    #df = scaler.transform(df)
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indices = [i for i,x in enumerate(list((df==0).all())) if x]
    df= df.drop(df.columns[noint_indices], axis=1)
    
    return df
X = add_interactions(X)
X.shape

In [7]:
###Seperate out the training and testing dataset

X.index = Y.index
X['is_promoted'] = Y

X_validation = X[54808:]
X = X[0:54808]

(78298, 830)

In [ ]:
X_validation.shape

In [8]:
Y = X.is_promoted
X = X.drop(['is_promoted'],1)
 

seed = 2
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [ ]:
###Hypertune the model intensively with a 5 fold cross validation strategy.
##first grid search for max_depth,min_child_weight then fix those and search for rest in same fashion
### parameter scale_pos_weight is quite important in case of imbalanced dataset

from sklearn.model_selection import GridSearchCV
param_test2b = {
  #'min_child_weight':[5,6],
  #'max_depth': range(3,10,2),
  #'n_estimators':[150,200,300,400],
  #'scale_pos_weight':[1,2,3,4],
  #'colsample_bytree':[0.7,0.8], 
  #'subsample':[0.7,0.8],
  #'gamma':[0,0.2.0.4]
    
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=150, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=3,seed=27), 
 param_grid = param_test2b, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(X_train, y_train)
print(gsearch2b.grid_scores_)
print("gsearch2b.best_params_",gsearch2b.best_params_)
print("gsearch2b.best_score_",gsearch2b.best_score_)


In [ ]:
###Train the model with the best params
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)
modelXg.fit(X_train, y_train)
y_xg = modelXg.predict(X_test)


In [9]:
## Determine whether your model is overfitting or not , with the help of ROC.
print(confusion_matrix(y_test, y_xg))
predictions = [value for value in y_xg]
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Accuracy_score: %.2f%% on test dataset" % (accuracy * 100.0))
print("precision_score: %.2f%% on test dataset" % (precision * 100.0))
print("recall_score: %.2f%% on test dataset" % (recall * 100.0))
print("f1_score: %.2f%% on test dataset" % (f1 * 100.0))
print("roc_auc test set", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))
print("roc_auc training set", roc_auc_score(y_train, model.predict_proba(X_train)[:,1]))

In [ ]:
##Once the model is ready, train the model on entire dataset.
modelXg = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, 
                      gamma=0.4,nthread=4, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)

modelXg.fit(X, Y)


In [ ]:
'''
##find the useful cols
usefulcols = []
for i in sorted( model.booster().get_fscore().items(), key=lambda x: x[1], reverse=True)[:100]:
    usefulcols.append(i[0])
print usefulcols
'''

In [ ]:
X = X_validation.drop(['is_promoted'],1)
dfvalidation = X_validation.drop(['is_promoted'],1)
y_valid = modelXg.predict(dfvalidation)
submission = dfvalidation.copy()
submission['is_promoted'] = y_valid
submission['is_promoted'].value_counts()

In [ ]:
submission = submission[['is_promoted']]
submission.reset_index(inplace=True)
submission.to_csv("D:\\analyticsvidhyahackathon\\solution.csv",index=False)s

In [2]:
sub = pd.read_csv("solution_18sept.csv")

In [3]:
private = pd.read_csv("private_W1BajhK.csv")
private.columns = ['employee_id', 'actual']
private = private.merge(sub, on = "employee_id", how="left")

In [4]:
from sklearn.metrics import f1_score

In [5]:
f1_score(private['actual'], private['is_promoted'])

0.5346232179226069